# Multiclass Classification
Using Logistic Regression with multiple categories

# 1. Intro to the data
The dataset we will be working with contains information on various cars. For each car we have information about the technical aspects of the vehicle such as the motor's displacement, the weight of the car, the miles per gallon, and how fast the car accelerates. Using this information we will predict the origin of the vehicle, either North America, Europe, or Asia. We can see, that unlike our previous classification datasets, we have three categories to choose from, making our task slightly more challenging.

The dataset is hosted by the University of California Irvine on their machine learning repository. As a side note, the UCI Machine Learning repository contains many small datasets which are useful when getting your hands dirty with machine learning.

In [64]:
import pandas as pd
cars = pd.read_csv('auto.csv')

print(cars.head())

    mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
0  18.0          8         307.0         130    3504          12.0    70   
1  15.0          8         350.0         165    3693          11.5    70   
2  18.0          8         318.0         150    3436          11.0    70   
3  16.0          8         304.0         150    3433          12.0    70   
4  17.0          8         302.0         140    3449          10.5    70   

   origin  
0       1  
1       1  
2       1  
3       1  
4       1  


In [65]:
#print out unique values in 'origin' column. These will be the classes. 
unique_origin = cars['origin'].unique()
print(unique_origin)

[1 3 2]


# 2. Dummy Variables
For this dataset, categorical variables exist in three columns, cylinders, year, and origin. The cylinders and year columns must be converted to numeric values so we can use them to predict label origin. Even though the column year is a number, we’re going to treat them like categories. **The year 71 is unlikely to relate to the year 70 in the same way those two numbers do numerically, but rather just as two different labels. In these instances, it is always safer to treat discrete values as categorical variables.**

We must use dummy variables for columns containing categorical values. Whenever we have more than 2 categories, we need to create more columns to represent the categories. Since we have 5 different categories of cylinders, we could use 3, 4, 5, 6, and 8 to represent the different categories. We can split the column into separate binary columns:

- cyl_3 -- Does the car have 3 cylinders? 0 if False, 1 if True.
- cyl_4 -- Does the car have 4 cylinders? 0 if False, 1 if True.
- cyl_5 -- Does the car have 5 cylinders? 0 if False, 1 if True.
- cyl_6 -- Does the car have 6 cylinders? 0 if False, 1 if True.
- cyl_8 -- Does the car have 8 cylinders? 0 if False, 1 if True.

We can use the **pandas.get_dummies() function to return a Dataframe containing binary columns from the values in the cylinders column.** In addition, if we **set the prefix parameter to cyl,** Pandas will pre-pend the column names to match the style we'd like:
     
    dummy_df = pd.get_dummies(cars["cylinders"], prefix="cylin")
    
We then use the pandas.concat() function to add the columns from this Dataframe back to cars:
    
    cars = pd.concat([cars, dummy_df], axis=1)

Now it's your turn! Repeat the same process for the year column.

Instructions
- Use the pandas.get_dummies() function to create dummy values from the year column.
- Use the prefix attribute to prepend year to each of the resulting column names.
- Assign the resulting Dataframe to dummy_years.
- Use the pandas.concat() function to concatenate the columns from dummy_years to cars.
- Use the DataFrame.drop() method to drop the year and cylinders columns from cars.
- Display the first 5 rows of the new cars Dataframe to confirm.

In [66]:
#create dummy variables for cylinder column
dummy_cylinders = pd.get_dummies(cars['cylinders'],prefix='cylin') 
cars = pd.concat([cars,dummy_cylinders],axis=1)

#create dummy variables for yea column
dummy_years = pd.get_dummies(cars['year'],prefix='yr')
cars = pd.concat([cars,dummy_years],axis=1)

cars=cars.drop('cylinders',axis=1)
cars=cars.drop('year',axis=1)

print(cars.head())

    mpg  displacement  horsepower  weight  acceleration  origin  cylin_3  \
0  18.0         307.0         130    3504          12.0       1        0   
1  15.0         350.0         165    3693          11.5       1        0   
2  18.0         318.0         150    3436          11.0       1        0   
3  16.0         304.0         150    3433          12.0       1        0   
4  17.0         302.0         140    3449          10.5       1        0   

   cylin_4  cylin_5  cylin_6  ...    yr_73  yr_74  yr_75  yr_76  yr_77  yr_78  \
0        0        0        0  ...        0      0      0      0      0      0   
1        0        0        0  ...        0      0      0      0      0      0   
2        0        0        0  ...        0      0      0      0      0      0   
3        0        0        0  ...        0      0      0      0      0      0   
4        0        0        0  ...        0      0      0      0      0      0   

   yr_79  yr_80  yr_81  yr_82  
0      0      0      0  

# 3. Multiclass Classification
In previous missions, we explored binary classification, where there were only 2 possible categories, or classes. When we have 3 or more categories, we call the problem a multiclass classification problem. There are a few different methods of doing multiclass classification and in this mission, we'll focus on the one-versus-all method.

The one-versus-all method is a technique where we choose a single category as the Positive case and group the rest of the categories as the False case. We're essentially splitting the problem into multiple binary classification problems. For each observation, the model will then output the probability of belonging to each category.

To start let's split our data into a training and test set. We've randomized the cars Dataframe for you already to start things off and assigned the shuffled Dataframe to shuffled_cars.

In [67]:
#randomize the data set
import numpy as np
shuffled_rows = np.random.permutation(cars.index)
shuffled_cars = cars.iloc[shuffled_rows]

#split shuffled_cars into 2 DFs: 
#1. a train DF with 70% of the observations
#2. a test DF with the other 30% of observations
highest_train_row = int(cars.shape[0] * .70)
train = shuffled_cars.iloc[0:highest_train_row]
test = shuffled_cars.iloc[highest_train_row:]


# 4. Training a Multiclass Logisitic Regression Model
In the **one-vs-all approach**, we're essentially converting an n-class (in our case n is 3) classification problem into n binary classification problems. For our case, we'll need to train 3 models:

- A model where all cars built in North America are considered Positive (1) and those built in Europe and Asia are considered Negative (0).
- A model where all cars built in Europe are considered Positive (1) and those built in North America and Asia are considered Negative (0).
- A model where all cars built in Asia are labeled Positive (1) and those built in North America and Europe are considered Negative (0).

Each of these models is a binary classification model that will return a probability between 0 and 1. When we apply this model on new data, a probability value will be returned from each model (3 total). For each observation, we choose the label corresponding to the model that predicted the highest probability.

We'll use the dummy variables we created from the cylinders and year columns to train 3 models using the LogisticRegression class from scikit-learn.

**Instructions**:

For each value in unique_origins, train a logistic regression model with the following parameters:
- X: Dataframe containing just the cylinder & year binary columns.
- y: list (or Series) of Boolean values:
  - True if observation's value for origin matches the current iterator variable (this will be treated as 1 by the comp)
  - False if observation's value for origin doesn't match the current iterator variable (this will be treated as 0 by the comp)

Add each model to the models dictionary with the following structure:
- key: origin value (1, 2, or 3),
- value: relevant LogistcRegression model instance.

In [68]:
from sklearn.linear_model import LogisticRegression

unique_origins = cars["origin"].unique()
unique_origins.sort()


models = {}
features = [c for c in train.columns if c.startswith("cyl") or c.startswith("yr")]

for origin in unique_origins:
    model = LogisticRegression()
    
    X_train = train[features]
    y_train = train["origin"] == origin #this will effectively create binary classes, either the origin (1) or not (0)

    model.fit(X_train, y_train)
    models[origin] = model #put model for particular origin class in the 'models' dictionary

# 5. Testing the Models
Now that we have a model for each category, we can run our test dataset through the models and evaluate how well they performed.

**Instructions:**

For each origin value from unique_origins:
- Use the LogisticRegression predict_proba function to return the 3 lists of predicted probabilities for the test set and add to the testing_probs Dataframe.

In [69]:
testing_probs = pd.DataFrame(columns=unique_origins)

for origin in unique_origins:
    # Select testing features.
    X_test = test[features]   
    # Compute probability of observation being in the origin.
    testing_probs[origin] = models[origin].predict_proba(X_test)[:,1]

testing_probs.head()

,1,2,3
0,0.968770,0.021399,0.034548
1,0.881603,0.051132,0.089585
2,0.278323,0.451925,0.269761
3,0.366557,0.205820,0.353144
4,0.955928,0.039920,0.026088


# 6. Choose the Origin
Now that we trained the models and computed the probabilities in each origin we can classify each observation. To classify each observation we want to select the origin with the highest probability of classification for that observation.

While each column in our dataframe testing_probs represents an origin we just need to choose the one with the largest probability. We can use the Dataframe **method .idxmax()** to return a Series where each value corresponds to the column or where the maximum value occurs for that observation. We need to make sure to set the axis paramater to 1 since we want to calculate the maximum value across columns. Since each column maps directly to an origin the resulting Series will be the classification from our model.

**Instructions:**
- Classify each observation in the test set using the testing_probs Dataframe.
- Assign the predicted origins to predicted_origins and use the print function to display it.


In [70]:
predicted_origins = testing_probs.idxmax(axis=1)
print(predicted_origins[0:10])

0    1
1    1
2    2
3    1
4    1
5    1
6    2
7    1
8    2
9    1
dtype: int64


# 7. Conclusion
In this mission, we learned the basics of extending logistic regression to work for multi-class classification problems. In the next mission, we'll dive into more intermediate linear regression concepts.